# Welcome to our main file ^^
## Before you get started, please make sure...
1. You have done generating bitstream in Vivado.
2. Upload `.hwh` and `.bit` file using whatever method to this folder.

## 0. Setting Up

#### Cell Description
This cell is used to set values to be execuded.<br>

Design Files can be found at: <br>
* .bit file:<br>`{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit`

* .hwh file:<br>`{ project_location }/{project_name}.gen/{Design Name 1}/bd/{Design Name 2}/hw_handoff/design_1.hwh`<br>

__Note:__ The design files whose filename ends with `.bit` or `.hwh` __must be renamed to be identical__.

In [1]:
# Design File. More detail below.
design_name = "top_warpper.bit"

# AXI Address
Setting_Kernal_Size_AXI_Address     = 0b1000
Setting_Ofmap_Ifmap_Channel_Address = 0b0000
Setting_Stride_Ofmap_Width_Address  = 0b0100
Write_Weight_Address                = 0b0000
Compute_Start_Address               = 0b0000
Getting_Return_Address              = 0b1100

# Preset Values
FUNC         = 0
IFMAPS_WIDTH = 7
IFMAPS_HIGHT = 7
IFMAPS_CH    = 7
WEIGHT_WIDTH = 3
WEIGHT_HIGHT = 3
WEIGHT_NUM   = 3
STRIDE       = 1

# Instruction
INST_WRITE_WEIGHT = 12
INST_COMPUTE      = 87
INST_LOADIFMAPS   = 88

#### Cell Description
This cell is to calcualte the real number (WDATA of AXI) to be transmitted.<br>

In [2]:
# Calculate WDATAs

# Set Kernal Size
Kernal_Size_WDATA = 0b00001 << WEIGHT_WIDTH

# Set Output Feature Map and Input Feature Map Channel Number
Ofmaps_Ifmaps_Channel_WDATA = (WEIGHT_NUM << 12 | IFMAPS_CH) << 8

# Set Stride & Output Feature Map Width
OFMAPS_WIDTH = round(((IFMAPS_WIDTH - WEIGHT_WIDTH) / (STRIDE + 1)));
Stride_Ofmaps_Func_WDATA = FUNC | OFMAPS_WIDTH << 2 | STRIDE << 12

#### Cell Description
This cell is used for import required packages. Also burn the `.bit` and `.hwh` file onto the PL side.

__Note:__ The filename of these two files __must be identical__.<br>

Only the `.bit` file needs to be added. Overlay constructor will find the `.hwh` file itself

In [3]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

import numpy as np

# Only the .bit file is needed.
ol = Overlay(design_name)

## 1. Sending Command & Preset Values

#### Cell Description

This cell is used to send AXI Lite signal to PL side.<br>
MMIO API Reference: [here](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)

In [4]:
mmio = MMIO(0x40000000, 0x1000) # allocate (base address, length)

# Set Kernal Size
mmio.write_reg(Setting_Kernal_Size_AXI_Address, Kernal_Size_WDATA)

# Set Output Feature Map and Input Feature Map Channel Number
mmio.write_reg(Setting_Ofmap_Ifmap_Channel_Address, Ofmaps_Ifmaps_Channel_WDATA)

# Set Stride & Output Feature Map Width
mmio.write_reg(Setting_Stride_Ofmap_Width_Address, Stride_Ofmaps_Func_WDATA)

# Send Start Writing Weight
mmio.write_reg(Write_Weight_Address, INST_WRITE_WEIGHT)

# Writing Weight
# To be continued...

# Read Return Value
ret = 0
print('ret = {}. Getting return value.'.format(ret), end = '')
while ret != 1:
    ret = mmio.read(Getting_Return_Address)
    print('.', end = '')
print('\nret = {}. Write weight done.'.format(ret))

ret = 0. Getting return value..
ret = 1. Write weight done.


## 2. Start Compute
### Work In Progress..

#### Cell Description

Set up DMA

In [5]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel